
# Visualizing GitHub Data Using APIs and Plotly

In this notebook, we will explore how to use data from GitHub's API to create interactive visualizations that highlight the most popular Python projects on the platform. By leveraging the power of APIs and data visualization libraries like Plotly, we can dynamically retrieve, process, and display up-to-date information about trending repositories.

## Introduction to APIs
An **Application Programming Interface (API)** allows applications to request and interact with data from a service using specific URLs. APIs provide structured responses in formats such as JSON or CSV, making it easy for programs to handle external data. Many modern applications, including those integrating social media or financial data, rely on APIs for real-time updates.

## Overview of Git and GitHub
GitHub is a widely used platform for hosting and collaborating on code projects, built on top of Git, a distributed version control system. Developers can manage project changes, collaborate efficiently, and track their progress using Git. GitHub repositories store project files, collaboration history, bug reports, and other details. Users can “star” repositories to bookmark and show support for projects they find valuable. In this notebook, we will analyze GitHub data to visualize the most-starred Python projects.

### Making API Calls to GitHub
GitHub’s API allows developers to access various types of project information. In this project, we’ll make API calls to gather data about Python repositories sorted by their star counts. For example, a request to:  
`https://api.github.com/search/repositories?q=language:python+sort:stars`  
retrieves details about the most popular Python repositories based on stars.

### Project Goals
- Use the **requests** library to make API calls to GitHub.
- Extract and process data about repository names, star counts, and descriptions e.t.c.
- Visualize the data using **Plotly**, creating an interactive bar chart that highlights the most-starred Python projects.

This project demonstrates how to build a self-contained data visualization tool using current data. By updating automatically with each API call, the visualizations stay current, reflecting the latest trends and popular repositories on GitHub. Let’s get started!


## Installing Requests
The Requests package allows a Python program to easily request information 
from a website and examine the response. Use pip to install Requests:
```python
$ python -m pip install --user requests
```
If you use a command other than python to run programs or start a terminal session, such as python3, your command will look like this:
```python
$ python3 -m pip install --user requests
```


### Processing an API Response
Now we’ll write a program to automatically issue an API call and process 
the results:

This section explains how to make an API request to GitHub and handle the response in Python. First, we import the `requests` module, set up the URL for the API call, and specify a query filter for Python repositories with more than 10,000 stars. We also define headers to explicitly use version 3 of the GitHub API and request the response in JSON format. Using `requests.get()`, we send the request and store the result in the variable `r`. We then check the `status_code` (200 indicates success) to confirm the request was successful. Finally, we use the `json()` method to convert the response into a Python dictionary and inspect its keys. This process sets the foundation for extracting and visualizing the data.

In [9]:

import requests
# Make an API call and check the response.
url = "https://api.github.com/search/repositories"
url += "?q=language:python+sort:stars+stars:>10000"
headers = {"Accept": "application/vnd.github.v3+json"}
r = requests.get(url, headers=headers)
print(f"Status code: {r.status_code}")

# Convert the response object to a dictionary.
response_dict = r.json()
# Process results.
print(response_dict.keys())

Status code: 200
dict_keys(['total_count', 'incomplete_results', 'items'])


## Working with the Response Dictionary

Once the API response is converted into a dictionary, we can explore its data. First, we check the total number of Python repositories returned using the `'total_count'` key and verify if the results are complete by inspecting `'incomplete_results'`. The `'items'` key contains a list of dictionaries, each representing a repository. We assign this list to `repo_dicts` and print its length to see how many repositories were retrieved. To examine the details of an individual repository, we extract the first dictionary, `repo_dict`, and print its number of keys and their names to understand the structure of the available data.


In [30]:
# Convert the response object to a dictionary.
response_dict = r.json()

print(f"Total repositories: {response_dict['total_count']}")
print(f"Complete results: {not response_dict['incomplete_results']}")
# Explore information about the repositories.
repo_dicts = response_dict['items']
print(f"Repositories returned: {len(repo_dicts)}")
# Examine the first repository.
repo_dict = repo_dicts[0]
print(f"\nKeys: {len(repo_dict)}")
for key in sorted(repo_dict.keys()):
 print(key)


Total repositories: 604
Complete results: True
Repositories returned: 30

Keys: 80
allow_forking
archive_url
archived
assignees_url
blobs_url
branches_url
clone_url
collaborators_url
comments_url
commits_url
compare_url
contents_url
contributors_url
created_at
default_branch
deployments_url
description
disabled
downloads_url
events_url
fork
forks
forks_count
forks_url
full_name
git_commits_url
git_refs_url
git_tags_url
git_url
has_discussions
has_downloads
has_issues
has_pages
has_projects
has_wiki
homepage
hooks_url
html_url
id
is_template
issue_comment_url
issue_events_url
issues_url
keys_url
labels_url
language
languages_url
license
merges_url
milestones_url
mirror_url
name
node_id
notifications_url
open_issues
open_issues_count
owner
private
pulls_url
pushed_at
releases_url
score
size
ssh_url
stargazers_count
stargazers_url
statuses_url
subscribers_url
subscription_url
svn_url
tags_url
teams_url
topics
trees_url
updated_at
url
visibility
watchers
watchers_count
web_commit_signoff_r

In this section, we extract and print key details from the first repository’s dictionary. We start with the repository's name, followed by accessing the nested `owner` dictionary to retrieve the owner's login name. We then print the number of stars the project has, the URL of the repository, its creation date, last update date, and description. This provides a quick summary of the repository's key attributes and helps understand the structure of the data returned by the API.

In [16]:
print("\nSelected information about first repository:")
print(f"Name: {repo_dict['name']}")
print(f"Owner: {repo_dict['owner']['login']}")
print(f"Stars: {repo_dict['stargazers_count']}")
print(f"Repository: {repo_dict['html_url']}")
print(f"Created: {repo_dict['created_at']}")
print(f"Updated: {repo_dict['updated_at']}")
print(f"Description: {repo_dict['description']}")


Selected information about first repository:
Name: public-apis
Owner: public-apis
Stars: 325379
Repository: https://github.com/public-apis/public-apis
Created: 2016-03-20T23:49:42Z
Updated: 2025-01-22T16:45:04Z
Description: A collective list of free APIs


We can see that the most-starred Python project on GitHub as of this 
writing is public-apis. Its owner is an organization with the same name, and 
it has been starred by more thon 300,000 GitHub users. We can see the URL 
for the project’s repository, its creation date of March 2016, and that it was 
updated recently. Additionally, the description tells us that public-apis contains a list of free APIs that programmers might be interested in.

## Summarizing the Top Repositories

To prepare for visualizing multiple repositories, we loop through each repository dictionary and print key details. First, we display an introductory message for context. Then, iterating over the `repo_dicts` list, we extract and print the repository’s name, owner’s login, star count, GitHub URL, and description. This step ensures we gather and inspect data for all repositories, not just the first one, enabling us to include comprehensive information in the final visualization. Some interesting projects appear in these results, and it might be worth 
looking at a few.


In [17]:


# Explore information about the repositories.
repo_dicts = response_dict['items']
print(f"Repositories returned: {len(repo_dicts)}")
print("\nSelected information about each repository:")
for repo_dict in repo_dicts:
 print(f"\nName: {repo_dict['name']}")
 print(f"Owner: {repo_dict['owner']['login']}")
 print(f"Stars: {repo_dict['stargazers_count']}")
 print(f"Repository: {repo_dict['html_url']}")
 print(f"Description: {repo_dict['description']}")

Repositories returned: 30

Selected information about each repository:

Name: public-apis
Owner: public-apis
Stars: 325379
Repository: https://github.com/public-apis/public-apis
Description: A collective list of free APIs

Name: system-design-primer
Owner: donnemartin
Stars: 286668
Repository: https://github.com/donnemartin/system-design-primer
Description: Learn how to design large-scale systems. Prep for the system design interview.  Includes Anki flashcards.

Name: awesome-python
Owner: vinta
Stars: 231286
Repository: https://github.com/vinta/awesome-python
Description: An opinionated list of awesome Python frameworks, libraries, software and resources.

Name: Python
Owner: TheAlgorithms
Stars: 196625
Repository: https://github.com/TheAlgorithms/Python
Description: All Algorithms implemented in Python

Name: AutoGPT
Owner: Significant-Gravitas
Stars: 170753
Repository: https://github.com/Significant-Gravitas/AutoGPT
Description: AutoGPT is the vision of accessible AI for everyone, t

## Monitoring API Rate Limits
Most APIs have rate limits, which means there’s a limit to how many requests 
you can make in a certain amount of time. To see if you’re approaching 
GitHub’s limits, enter https://api.github.com/rate_limit into a web browser. You 
should see a response that begins like this:
```js
{"resources":{"core":{"limit":60,"remaining":60,"reset":1737580741,"used":0,"resource":"core"},"graphql":{"limit":0,"remaining":0,"reset":1737580741,"used":0,"resource":"graphql"},"integration_manifest":{"limit":5000,"remaining":5000,"reset":1737580741,"used":0,"resource":"integration_manifest"},"search":{"limit":10,"remaining":10,"reset":1737577201,"used":0,"resource":"search"}},"rate":{"limit":60,"remaining":60,"reset":1737580741,"used":0,"resource":"core"}}
```

In this section, we check the rate limit for GitHub's search API to manage request usage effectively. The limit allows up to 60 requests per minute, and we monitor how many requests remain for the current period. The `"reset"` key provides the Unix time when the quota will reset. If the limit is exceeded, a response indicates the restriction, requiring a wait until the reset time. 

Many APIs require an access token for higher limits, but GitHub’s API allows basic usage without one, though registering for a token can significantly increase your request allowance.

## Visualizing Repositories Using Plotly
Here, we create a visualization to display the relative popularity of Python projects on GitHub using an interactive bar chart. Each bar’s height represents the number of stars a project has received, while the label links directly to the project’s GitHub page. This approach provides a clear, dynamic way to explore top Python repositories and their popularity at a glance.

In [18]:
import requests
import plotly.express as px
# Make an API call and check the response.
url = "https://api.github.com/search/repositories"
url += "?q=language:python+sort:stars+stars:>10000"
headers = {"Accept": "application/vnd.github.v3+json"}
r = requests.get(url, headers=headers)
print(f"Status code: {r.status_code}")
# Process overall results.
response_dict = r.json()
print(f"Complete results: {not response_dict['incomplete_results']}")
# Process repository information.
repo_dicts = response_dict['items']
repo_names, stars = [], []
for repo_dict in repo_dicts:
 repo_names.append(repo_dict['name'])
 stars.append(repo_dict['stargazers_count'])
# Make visualization.
fig = px.bar(x=repo_names, y=stars)
fig.show()

Status code: 200
Complete results: True


## Styling the Chart

We enhance the visualization by adding a descriptive title and axis labels using `px.bar()`. Next, we customize the plot's appearance with `fig.update_layout()`, adjusting font sizes for readability—setting the title font size to 28 and axis title sizes to 20. Plotly’s naming conventions, using underscores to link chart element attributes, make customization intuitive and consistent. These improvements ensure the chart is both informative and visually appealing.

In [19]:

# Make visualization.
title = "Most-Starred Python Projects on GitHub"
labels = {'x': 'Repository', 'y': 'Stars'}
fig = px.bar(x=repo_names, y=stars, title=title, labels=labels)
fig.update_layout(title_font_size=28, xaxis_title_font_size=20, yaxis_title_font_size=20)
fig.show()

## Adding Custom Tooltips

To create custom tooltips for our visualization, we define an empty list, `hover_texts`, to store descriptive text for each project. In the data processing loop, we extract the project owner and description. Using HTML formatting, we combine these with a line break (`<br />`) to create a multi-line label, appending each to `hover_texts`. By passing this list to the `hover_name` argument in `px.bar()`, Plotly uses these custom labels for tooltips, displaying detailed information when users hover over a bar. This enhances the interactive experience by providing context-rich insights for each project.

In [24]:


# Process repository information.
repo_dicts = response_dict['items']
repo_names, stars, hover_texts = [], [], []
for repo_dict in repo_dicts:
 repo_names.append(repo_dict['name'])
 stars.append(repo_dict['stargazers_count'])
 # Build hover texts.
 owner = repo_dict['owner']['login']
 description = repo_dict['description']
 hover_text = f"{owner}<br />{description}"
 hover_texts.append(hover_text)
# Make visualization.
title = "Most-Starred Python Projects on GitHub"
labels = {'x': 'Repository', 'y': 'Stars'}
fig = px.bar(x=repo_names, y=stars, title=title, labels=labels , hover_name=hover_texts) 

fig.update_layout(title_font_size=28, xaxis_title_font_size=20, yaxis_title_font_size=20)
fig.show()

## Adding Clickable Links

To add clickable links to our chart in Plotly, we modify the `repo_names` list to `repo_links` to better reflect its purpose. We extract each repository’s URL from `repo_dict` and store it in a variable, `repo_url`. Using the HTML anchor tag (`<a href='URL'>link text</a>`), we generate a hyperlink with the repository name as the link text and append it to `repo_links`. By passing `repo_links` as the x-axis values in `px.bar()`, we create an interactive chart where users can click on project names to visit their GitHub pages, enhancing both the functionality and engagement of our visualization.

In [28]:

# Process repository information.
repo_dicts = response_dict['items']
repo_links, stars, hover_texts = [], [], []
for repo_dict in repo_dicts:
    # Turn repo names into active links.
    repo_name = repo_dict['name']
    repo_url = repo_dict['html_url']
    repo_link = f"<a href='{repo_url}'>{repo_name}</a>"
    repo_links.append(repo_link)
    stars.append(repo_dict['stargazers_count'])
    
    # Add repository description to hover texts.
    description = repo_dict.get('description', 'No description available')
    hover_text = f"{repo_name}: {description}"
    hover_texts.append(hover_text)

# Make visualization.
title = "Most-Starred Python Projects on GitHub"
labels = {'x': 'Repository', 'y': 'Stars'}
fig = px.bar(x=repo_links, y=stars, title=title, labels=labels, hover_name=hover_texts)
fig.update_layout(title_font_size=28, xaxis_title_font_size=20, yaxis_title_font_size=20)
fig.show()


## Customizing Marker Colors

In Plotly, traces represent individual collections of data points on a chart, and you can customize them using the `update_traces()` method. To modify the appearance of the bars, we change their color to a darker shade of blue using `'SteelBlue'`, a named CSS color. We also adjust the bars’ transparency by setting the `opacity` parameter to `0.6`. This makes the bars slightly see-through, enhancing the visual appeal and readability of overlapping elements. The `marker_` prefix allows fine-tuning of properties like color, opacity, and size for visual elements on the chart.

In [29]:
fig.update_layout(title_font_size=28, xaxis_title_font_size=20,
 yaxis_title_font_size=20)
fig.update_traces(marker_color='SteelBlue', marker_opacity=0.6)
fig.show()

## Conclusion
In this project, we explored GitHub’s API to gather data on the most popular Python repositories and created an interactive bar chart visualization using Plotly. The chart displays the number of stars each project has received, with clickable project names linking to their respective GitHub pages. We customized tooltips to show project descriptions and owners, enhanced visual styling by adjusting the bar colors and transparency, and demonstrated how to update chart elements for improved readability. This project illustrates how APIs and data visualization tools can be combined to present meaningful, dynamic insights from real-time data.